In [47]:
# Most of this code is taken from the DeepLearning.ai's Natural Language Processing with Probabilistic Models course assignment ('Autocorrect')
# and it is reshaped by centering another txt file as a vocabulary
# The text is from Oguz Atay's book: 'Tehlikeli Oyunlar'

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize #for tokenization
from nltk.stem import PorterStemmer #for stemming
import snowballstemmer #We also import this one for "Turkish"
from nltk.corpus import stopwords 
import numpy as np
import pandas as pd
from collections import Counter

In [48]:
def processing_data(file_name):
    
    with open(file_name, encoding="utf8") as f:
        file_name_d = f.read()
    file_name_d_lower = file_name_d.lower() #.lower() for lowering the letters
    # tokenize the text word by word
    words = word_tokenize(file_name_d_lower)
    
    return words

In [49]:
word_list = processing_data('oguzatay.txt')
vocab = set(word_list) # as a vocabulary we choose Oguz Atay : Tehlikeli Oyunlar
print(f"The first ten words in the text are: \n{word_list[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['oğuz', 'atay', 'tehlikeli', 'oyunlar', 'bütün', 'eserleri/', 'i̇stanbul', 'tehli̇keli̇', 'oyunlar', 'oğuz']
There are 27928 unique words in the vocabulary.


In [50]:
# We are going to define get_count function for counting how many times a word appear in the corpus

In [51]:
def get_count(word_list):
    word_count_dict = {} #fill this with word counts
    word_count_dict = Counter(word_list)
    return word_count_dict

In [52]:
word_count_dict = get_count(word_list)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'başlamak' is {word_count_dict.get('başlamak',0)}")

There are 27928 key values pairs
The count for the word 'başlamak' is 4


In [53]:
# We are going to implement get_probs function which gives you the probability that a word occurs in a sample.

In [54]:
def get_probs(word_count_dict):
    probs = {}
    cardinality = sum(word_count_dict.values())
    for key in word_count_dict.keys():
        probs[key] = word_count_dict[key]/cardinality
    return probs

In [55]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('başlamak') is {probs['başlamak']:.6f}")

Length of probs is 27928
P('başlamak') is 0.000027


In [56]:
def delete_letter(word, verbose=False):
    
    delete_l = []
    split_l = []

    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    delete_l = [(L+R[1:]) for L, R in split_l if R]

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [57]:
def switch_letter(word, verbose=False):

    switch_l = []
    split_l = []

    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    switch_l = [(L + R[1] + R[0] + R[2:]) for L, R in split_l if len(R)>=2]

    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [58]:
def replace_letter(word, verbose=False):

    letters = 'abcçdefgğhıijklmnoöpqrsştuüvyz'
    replace_l = []
    split_l = []

    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    replace_l = [(L+C+R[1:]) for L, R in split_l if len(R)>=1 for C in letters]
    replace_set = set(replace_l)
    replace_set.discard(word)

    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [59]:
def insert_letter(word, verbose=False):
    
    letters = 'abcçdefgğhıijklmnoöpqrsştuüvyz'
    insert_l = []
    split_l = []
    
    ### START CODE HERE ###
    split_l = [(word[:i],word[i:]) for i in range(len(word)+1)]
    insert_l = [(L+c+R) for L,R in split_l if 1 for c in letters]
    
    ### END CODE HERE ###

    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [60]:
def edit_one_letter(word, allow_switches = True):

    edit_one_set = set()
    
    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))

    return edit_one_set

In [61]:
def edit_two_letters(word, allow_switches = True):

    edit_two_set = set()

    edit_one = edit_one_letter(word,allow_switches=allow_switches)
    for w in edit_one:
        edit_two = edit_one_letter(w,allow_switches=allow_switches)
        edit_two_set.update(edit_two)

    return edit_two_set

In [62]:
def get_corrections(word, probs, vocab, n=2, verbose = False):

    suggestions = []
    n_best = []

    suggestions = list((word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab))
    
    suggestions = list(reversed(suggestions))
    for w in suggestions:
        n_best.append([w,probs[w]])    

    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [63]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    
    # use deletion and insert cost as  1
    m = len(source) 
    n = len(target) 
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1,m+1): # Replace None with the proper range
        D[row,0] = D[row-1,0] + del_cost
        
    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1,n+1): # Replace None with the proper range
        D[0,col] = D[0,col-1] + ins_cost
        
    # Loop through row 1 to row m, both inclusive
    for row in range(1,m+1): 
        
        # Loop through column 1 to column n, both inclusive
        for col in range(1,n+1):
            
            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost
            
            # Check to see if source character at the previous row
            # matches the target character at the previous column, 
            if source[row-1] == target[col-1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0
                
            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row,col] = min(D[row-1,col] + del_cost , D[row,col-1] + ins_cost, D[row-1,col-1] + r_cost)
          
    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m,n]
    
    return D, med

In [64]:
source =  'anlatı'
target = 'anlattı'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  1 

   #  a  n  l  a  t  t  ı
#  0  1  2  3  4  5  6  7
a  1  0  1  2  3  4  5  6
n  2  1  0  1  2  3  4  5
l  3  2  1  0  1  2  3  4
a  4  3  2  1  0  1  2  3
t  5  4  3  2  1  0  1  2
ı  6  5  4  3  2  1  2  1


In [65]:
my_word = 'başanmak' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

# CODE REVIEW COMMENT: using "tmp_corrections" insteads of "cors". "cors" is not defined
print(f"data type of corrections {type(tmp_corrections)}")

entered word =  başanmak 
suggestions =  ['boşanmak']
word 0: boşanmak, probability 0.000007
data type of corrections <class 'list'>


In [66]:
source =  'başanmak'
target = 'boşanmak'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  2 

   #  b  o  ş  a  n  m  a  k
#  0  1  2  3  4  5  6  7  8
b  1  0  1  2  3  4  5  6  7
a  2  1  2  3  2  3  4  5  6
ş  3  2  3  2  3  4  5  6  7
a  4  3  4  3  2  3  4  5  6
n  5  4  5  4  3  2  3  4  5
m  6  5  6  5  4  3  2  3  4
a  7  6  7  6  5  4  3  2  3
k  8  7  8  7  6  5  4  3  2
